In [1]:
import pandas as pd
import numpy as np
import os
import json
import ast
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import chain
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
english_stop_words = list(stopwords.words('english'))
import gensim.corpora as corpora
import gensim
from gensim.utils import simple_preprocess
from pprint import pprint
import pyLDAvis.gensim
import pickle
import pyLDAvis
import time
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pi96a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pi96a\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
path = "C:/Drive D/Datasets/Aminer Data"
os.chdir(path)

In [3]:
topic_words_df = pd.read_csv("new_dblp-ref-3 LDA topic words.csv")

In [4]:
topic_words_df.drop(['Unnamed: 0'], axis = "columns", inplace = True)

In [5]:
doc_topic_df = pd.read_csv("new_dblp-ref-3 doc-topic-df.csv")

In [6]:
doc_topic_df.shape

(35174, 2)

In [7]:
doc_topic_df

,Unnamed: 0,Topic
0,0,1
1,1,2
2,2,1
3,3,5
4,4,4
...,...,...
35169,35169,0
35170,35170,4
35171,35171,5
35172,35172,0


In [8]:
%%time
# Removing all the Null values from the dataset
df = pd.read_json("new_dblp-ref-3.json")
# drop all NA values from the dataset
print("Before Dropping NA : ", df.shape)
df = df.dropna().reset_index(drop = True)
print("After Dropping NA : ", df.shape)

Before Dropping NA :  (79007, 8)
After Dropping NA :  (35174, 8)
CPU times: total: 672 ms
Wall time: 1.06 s


In [9]:
df = pd.concat([doc_topic_df[['Topic']], df], axis = "columns")

### String Preprocessing

In [10]:
# generating content column for each row in dataset
df["Content"] = df["abstract"].str.lower() + df["title"].str.lower()

In [11]:
sub_df = df.sample(5000, random_state = 23).reset_index(drop = True)

In [12]:
sub_df.head(5)

,Topic,abstract,authors,n_citation,references,title,venue,year,id,Content
0,1,The open-source software project OpenPhase all...,"[Marvin Tegeler, Oleg Shchyglo, Reza Darvishi ...",0,"[35d0c371-6a04-49a1-822d-16365ec0b4ef, b3dd5f8...",Parallel multiphase field simulations with Ope...,Computer Physics Communications,2017,a4f44262-95d0-4550-a0bf-56b8cf55b2e2,the open-source software project openphase all...
1,3,Wireless sensor networks with radio frequency ...,"[Vojislav B. Misic, Jelena V. Misic, Mohammad ...",0,"[15cb7fd8-134a-4c30-a579-286a685a320d, 1aee33c...",Optimum Zoning in RF-Recharged Sensor Networks,vehicular technology conference,2016,492121d5-7ef4-490b-956b-610ec45a38e8,wireless sensor networks with radio frequency ...
2,4,This research was performed to investigate the...,"[Majid Memarian Sorkhabi, Javad Frounchi, Parv...",0,"[05be8ba6-2e04-4f03-a310-b5dd9997aa4e, 26c5aec...",Deep-Brain Transcranial Stimulation: A Novel A...,IEEE Access,2017,fee7ef89-1665-4b8b-927d-b5fd80194002,this research was performed to investigate the...
3,1,The performance of stereophonic acoustic echo ...,"[Maria Luis Valero, Ilkay Yildiz, Edwin Maband...",0,"[1377b277-655a-4b44-9181-2e28c12aa5a4, 5528066...",Coherence-aware stereophonic residual echo est...,,2017,db0d91ec-9297-4493-8df2-9443a3929204,the performance of stereophonic acoustic echo ...
4,5,Web-based Social Networks (W-bSNs) have recent...,"[Yecely Aridaí Díaz-Beristain, Guillermo-de-Je...",0,[3c0c7536-7f11-4e63-bf6d-32d67050d8ef],Strategies for Growing User Popularity through...,advances in multimedia,2017,8e58438d-5c8f-4e4e-890b-ed71a8172a8b,web-based social networks (w-bsns) have recent...


In [13]:
%%time
# Creating stop words list
stop_words_extended_list = ["the", "of", "and", "a", "in", "to", "is", 
                            "?", "for", "we", "that", "-", "=", "with", ",", 
                            "on", "as", "by", "are", "this", "be", ".", 
                            "can", "an", "from", ")", "which", "it", "+", 
                            "each", "our", "at", "where", "has", "such", 
                            "a", "b", "c", "d", "e", "f", "g", "h", "i",
                            "j", "k", "l", "m", "n", "o", "p", "q", "r",
                            "s", "t", "u", "v", "w", "x", "y", "z", 
                            "0", "1", "2", "3", "4", "5", "6", "7", "8", 
                            "9", "et", "j.", "", "(", "a)", "b)"]
english_stop_words.extend(stop_words_extended_list)
english_stop_words = set(english_stop_words)

CPU times: total: 0 ns
Wall time: 0 ns


In [14]:
%%time
def remove_stopwords(row):
    new_string = []
    for word in row.split(" "):
        if word in english_stop_words:
            continue
        else:
            new_string.append(word)
    return " ".join(new_string)

sub_df["Non Stop Word Content"] = sub_df["Content"].apply(remove_stopwords)

CPU times: total: 46.9 ms
Wall time: 103 ms


In [15]:
%%time
wordnet_lemmatizer = WordNetLemmatizer()
def perform_lemmatization(row):
    new_string = []
    for word in row.split(" "):
        lemmatized_word = wordnet_lemmatizer.lemmatize(word)
        new_string.append(lemmatized_word)
    return " ".join(new_string)

sub_df["lemmatized Content"] = sub_df["Non Stop Word Content"].apply(perform_lemmatization)

CPU times: total: 2.08 s
Wall time: 2.41 s


In [16]:
# Count Vectorizer to see how many number of words are getting repeated across
#count = CountVectorizer(stop_words = 'english', min_df = 10)
#count_matrix = count.fit_transform(sub_df["lemmatized Content"])
#count_matrix.shape

In [17]:
# TFIDF Vectorizer
tfidf = TfidfVectorizer(stop_words = 'english', min_df = 5)
tfidf_matrix = tfidf.fit_transform(sub_df["lemmatized Content"])
tfidf_matrix.shape 

(5000, 7044)

In [18]:
# Computing the cosine similarity matrix for generated TFIDF matrix
cosine_similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

In [19]:
cosine_similarity_matrix.shape

(5000, 5000)

In [20]:
df.head()

,Topic,abstract,authors,n_citation,references,title,venue,year,id,Content
0,1,AdaBoost algorithm based on Haar-like features...,"[Zheng Xu, Runbin Shi, Zhihao Sun, Yaqi Li, Yu...",0,"[0a11984c-ab6e-4b75-9291-e1b700c98d52, 1f4152a...",A Heterogeneous System for Real-Time Detection...,high performance computing and communications,2016,001eef4f-1d00-4ae6-8b4f-7e66344bbc6e,adaboost algorithm based on haar-like features...
1,2,"In this paper, a kind of novel jigsaw EBG stru...","[Yufei Liang, Yan Zhang, Tao Dong, Shan-wei Lu]",0,[],A novel conformal jigsaw EBG structure design,international conference on conceptual structures,2016,002e0b7e-d62f-4140-b015-1fe29a9acbaa,"in this paper, a kind of novel jigsaw ebg stru..."
2,1,This paper studies the problem of using an aut...,"[Xiaodong Ai, Keyou You, Shiji Song]",0,"[1862a08a-08c6-4ab1-a214-8932bbd0d2d9, 7bcea2f...",A source-seeking strategy for an autonomous un...,"international conference on control, automatio...",2016,00352759-f0a7-4678-82ae-fed68c700da6,this paper studies the problem of using an aut...
3,5,This paper presents a methodology to analyze l...,"[Vincent Buntinx, Cyril Bornet, Frédéric Kaplan]",0,"[426b57a8-2e7d-498d-9a57-c09983ae0699, 6499d57...",Studying Linguistic Changes over 200 Years of ...,Frontiers in Digital Humanities,2017,01522369-3b88-4256-99d4-4e0c1de9f1ba,this paper presents a methodology to analyze l...
4,4,Container identification and recognition is st...,"[Ankit Verma, Monika Sharma, Ramya Hebbalagupp...",0,"[3e3a4245-eb62-4769-bc84-1001479df701, 42f6b97...",Automatic Container Code Recognition via Spati...,international conference on machine learning a...,2016,017440d5-6ba8-422a-bd7a-d59e3b9a4e77,container identification and recognition is st...


In [21]:
sub_df.head()

,Topic,abstract,authors,n_citation,references,title,venue,year,id,Content,Non Stop Word Content,lemmatized Content
0,1,The open-source software project OpenPhase all...,"[Marvin Tegeler, Oleg Shchyglo, Reza Darvishi ...",0,"[35d0c371-6a04-49a1-822d-16365ec0b4ef, b3dd5f8...",Parallel multiphase field simulations with Ope...,Computer Physics Communications,2017,a4f44262-95d0-4550-a0bf-56b8cf55b2e2,the open-source software project openphase all...,open-source software project openphase allows ...,open-source software project openphase allows ...
1,3,Wireless sensor networks with radio frequency ...,"[Vojislav B. Misic, Jelena V. Misic, Mohammad ...",0,"[15cb7fd8-134a-4c30-a579-286a685a320d, 1aee33c...",Optimum Zoning in RF-Recharged Sensor Networks,vehicular technology conference,2016,492121d5-7ef4-490b-956b-610ec45a38e8,wireless sensor networks with radio frequency ...,wireless sensor networks radio frequency recha...,wireless sensor network radio frequency rechar...
2,4,This research was performed to investigate the...,"[Majid Memarian Sorkhabi, Javad Frounchi, Parv...",0,"[05be8ba6-2e04-4f03-a310-b5dd9997aa4e, 26c5aec...",Deep-Brain Transcranial Stimulation: A Novel A...,IEEE Access,2017,fee7ef89-1665-4b8b-927d-b5fd80194002,this research was performed to investigate the...,research performed investigate ability transcr...,research performed investigate ability transcr...
3,1,The performance of stereophonic acoustic echo ...,"[Maria Luis Valero, Ilkay Yildiz, Edwin Maband...",0,"[1377b277-655a-4b44-9181-2e28c12aa5a4, 5528066...",Coherence-aware stereophonic residual echo est...,,2017,db0d91ec-9297-4493-8df2-9443a3929204,the performance of stereophonic acoustic echo ...,performance stereophonic acoustic echo cancele...,performance stereophonic acoustic echo cancele...
4,5,Web-based Social Networks (W-bSNs) have recent...,"[Yecely Aridaí Díaz-Beristain, Guillermo-de-Je...",0,[3c0c7536-7f11-4e63-bf6d-32d67050d8ef],Strategies for Growing User Popularity through...,advances in multimedia,2017,8e58438d-5c8f-4e4e-890b-ed71a8172a8b,web-based social networks (w-bsns) have recent...,web-based social networks (w-bsns) recently ex...,web-based social network (w-bsns) recently exp...


In [22]:
# search parameters
#search_parameter = input("Enter the keywords you want to search papers for: ")
search_parameter = "machines, computers, mathematics, studies, investigated"

In [23]:
keywords = [word.split(" ") for word in search_parameter.split(",")]
keywords = list(chain.from_iterable(keywords))
keywords = [word for word in keywords if word.isalpha()]
keywords

['machines', 'computers', 'mathematics', 'studies', 'investigated']

In [24]:
def return_search_topics(keywords):
    search_index = {}
    for col in topic_words_df.columns:
        intersection_number = len(set(keywords).intersection(set(topic_words_df[col].values)))
        if intersection_number > 0:
            search_index[col] = (intersection_number / len(set(topic_words_df[col].values))) * 100
    search_index = [int(index.split("_")[1]) for index in search_index.keys()]
    return search_index

In [25]:
search_index = return_search_topics(keywords)

In [26]:
search_index

[5]

In [27]:
# initial search results based on citations
sub_df[sub_df["Topic"].isin(search_index)].sort_values(["n_citation"], ascending = False).head(10)

,Topic,abstract,authors,n_citation,references,title,venue,year,id,Content,Non Stop Word Content,lemmatized Content
2956,5,Adaptive hypermedia is a relatively new direct...,[Peter Brusilovsky],3192,"[01c65f07-9d53-4844-8e25-ada6745a05d8, 13ca496...",Adaptive Hypermedia,,2001,8ab23f4b-0b87-42ed-9922-dbd10d9e845d,adaptive hypermedia is a relatively new direct...,adaptive hypermedia relatively new direction r...,adaptive hypermedia relatively new direction r...
3483,5,WordNet: Similarity is a freely available soft...,"[Ted Pedersen, Siddharth Patwardhan, Jason Mic...",1526,"[11d6d688-f542-4d4e-8a9e-50150c7f08e3, 4a23bcc...",WordNet: similarity - measuring the relatednes...,national conference on artificial intelligence,2004,ccd672d4-2f77-471e-abfc-cae8abd6ec16,wordnet: similarity is a freely available soft...,wordnet: similarity freely available software ...,wordnet: similarity freely available software ...
2709,5,Calendar applications for small handheld devic...,"[Benjamin B. Bederson, Aaron Clamage, Mary Cze...",242,"[1a6548e3-ac10-4a17-a70f-b5a95bf4866a, 2b49543...",DateLens: A fisheye calendar interface for PDAs,ACM Transactions on Computer-Human Interaction,2004,41547a1b-4659-4ae4-8530-29dee0dbaad5,calendar applications for small handheld devic...,calendar applications small handheld devices g...,calendar application small handheld device gro...
2353,5,Recent literature on sustainable logistics net...,"[J. Quariguasi Frota Neto, Grit Walther, J.M. ...",195,"[22720135-99b5-4fe4-81bb-2e07ce6e3794, 441bb34...",A methodology for assessing eco-efficiency in ...,European Journal of Operational Research,2009,4d1323bf-336a-4b07-80d5-8884df56d71b,recent literature on sustainable logistics net...,recent literature sustainable logistics networ...,recent literature sustainable logistics networ...
1404,5,Problem detection is the process by which peop...,"[Gary Klein, Rebecca M. Pliske, Beth Crandall,...",111,"[9a88b9ee-6610-445e-b2a4-73f889c8e944, c5e2141...",Problem detection,,2005,17c7d836-6a3e-444d-94c3-7e1bdfae9724,problem detection is the process by which peop...,problem detection process people first become ...,problem detection process people first become ...
2023,5,We argue that the analysis of agent/environmen...,"[Philip E. Agre, Ian Horswill]",110,"[2329ae1f-1062-4504-9ef1-45d9e5c0dd2e, 245d5a4...",Lifeworld analysis,,1997,456c0cab-a2d9-4527-9ecd-2fdaea24a67c,we argue that the analysis of agent/environmen...,argue analysis agent/environment interactions ...,argue analysis agent/environment interaction e...
3030,5,This paper argues that the current OO technolo...,"[Krzysztof Czarnecki, Ulrich W. Eisenecker]",70,"[1c915292-2f7b-4af7-9bc1-187001c58e95, 28c49b7...",Synthesizing Objects,european conference on object oriented program...,1999,376f54a3-be60-4fd1-a712-39ae2c4f03df,this paper argues that the current oo technolo...,paper argues current oo technology support reu...,paper argues current oo technology support reu...
3240,5,Using simple examples and informal discussions...,[Arjen K. Lenstra],62,"[11c5b635-0a99-483a-b927-86ee4624557b, 1382c60...",Integer Factoring,public key cryptography,2000,8980cde7-a038-4a5b-a776-cd72747cebc8,using simple examples and informal discussions...,using simple examples informal discussions art...,using simple example informal discussion artic...
3695,5,While mereotopology – the theory of boundaries...,"[Barry Smith, Berit Brogaard]",61,"[00ae2773-3978-4023-9b65-ac4559eef30b, 23f3b4a...",Quantum Mereotopology,,2002,ec79903d-b836-4038-83a8-efe2e469e00d,while mereotopology – the theory of boundaries...,"mereotopology – theory boundaries, contact sep...","mereotopology – theory boundaries, contact sep..."
1302,5,Typestate reflects how the legal operations on...,"[Roger Wolff, Ronald Garcia, Éric Tanter, Jona...",55,"[0d3aef95-b2e6-478d-ab68-e6f256b8cd8f, 0f35142...",Gradual typestate,european conference on object oriented program...,2011,c5bc2714-893e-42ce-b1d5-300c42758bac,typestate reflects how the

In [28]:
index_number = 2956

# get the details of the paper which matches the index number
selected_paper = sub_df[sub_df.index == index_number]
    
# get the pairwise similarity score of all the papers with the selected paper
sim_scores = list(enumerate(cosine_similarity_matrix[index_number]))

# sort the similar papers based on the similarity score
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

# get the scores of the 10 most similar papers
sim_scores = sim_scores[1:11]

# get the indices of similar papers
sim_indices = [i[0] for i in sim_scores]

sim_indices

[3950, 1603, 3971, 935, 324, 2775, 2617, 2922, 1891, 3103]

In [29]:
sim_scores

[(3950, 0.25138097893883854),
 (1603, 0.21453211799303623),
 (3971, 0.1954050963505293),
 (935, 0.1900651599221814),
 (324, 0.18537174500566417),
 (2775, 0.17437533780321773),
 (2617, 0.17065699654738753),
 (2922, 0.170515460459254),
 (1891, 0.1667262963257014),
 (3103, 0.1661442066337546)]

In [30]:
temp_df = sub_df.reset_index()
temp_df[temp_df['index'].isin(sim_indices)]

,index,Topic,abstract,authors,n_citation,references,title,venue,year,id,Content,Non Stop Word Content,lemmatized Content
324,324,5,Users of e-commerce sites often read reviews o...,"[Wenting Tu, David W. Cheung, Nikos Mamoulis]",0,"[089f3381-8ddd-4c66-a339-471c5d26b075, 0c2a835...",More focus on what you care about: Personalize...,Neurocomputing,2017,b07486b2-fd37-4052-b0ed-d31da0eaa60e,users of e-commerce sites often read reviews o...,users e-commerce sites often read reviews prod...,user e-commerce site often read review product...
935,935,0,This paper is concerned with the adaptive fini...,"[Lin Zhao, Yingmin Jia, Jinpeng Yu]",50,"[07d00f5d-7593-478f-89e9-c2490be3adb3, 0bd0673...",Adaptive finite-time bipartite consensus for s...,Systems & Control Letters,2017,08c0dfa5-4e4b-4cf5-bd22-c60b8d75906e,this paper is concerned with the adaptive fini...,paper concerned adaptive finite-time bipartite...,paper concerned adaptive finite-time bipartite...
1603,1603,5,The Traditional Malay Houses (TMH) in Malaysia...,"[Hafez Zainudin, Nurul Ain Haron, Saiful Hazmi...",0,[],Utilization of building information modeling (...,virtual systems and multimedia,2016,028dff88-2889-49d7-a49d-1d805a6e7f0d,the traditional malay houses (tmh) in malaysia...,traditional malay houses (tmh) malaysia consid...,traditional malay house (tmh) malaysia conside...
1891,1891,5,The area of population-based meta-heuristics h...,"[Frank Vanhoenshoven, Gonzalo Nápoles, Mathijs...",0,"[1dbfb3b6-6c4f-4abe-91c7-9d7cbce10e3d, 293971a...",Analyzing the impact of the adaptive clearing ...,,2016,8870c55f-28a5-4f9e-a49e-e85e49039788,the area of population-based meta-heuristics h...,area population-based meta-heuristics research...,area population-based meta-heuristics research...
2617,2617,5,In this work we propose a novel representation...,"[Jegatheesan Saravana Ganesh, Manish Gupta, Va...",0,"[22ed0c87-ae84-4b4a-9073-9502d95e2f39, 2c8cc23...",Improving Tweet Representations using Temporal...,european conference on information retrieval,2016,111b60bf-2350-4265-b4aa-6bc8264080ed,in this work we propose a novel representation...,work propose novel representation learning mod...,work propose novel representation learning mod...
2775,2775,5,"With the invention of fitness trackers, it has...","[Zae Myung Kim, Hyungrai Oh, Han-Gyu Kim, Chae...",50,"[20733ed2-84ec-4b8c-8ada-a525dfd8dfdf, 2904254...",Modeling long-term human activeness using recu...,BMC Medical Informatics and Decision Making,2017,6c518a8d-b9d0-4d53-a29b-6ecce4a76a16,"with the invention of fitness trackers, it has...","invention fitness trackers, possible continuou...","invention fitness trackers, possible continuou..."
2922,2922,4,Superpixels are perceptually meaningful atomic...,"[Peng Xu, Jie Li, Juan Yue, Xia Yuan]",0,"[031bbeb0-4dc1-46f9-9984-109c3b3eefae, 356fdf3...",Scale adaptive supervoxel segmentation of RGB-...,robotics and biomimetics,2016,7023eec3-bfe0-46de-a7f5-861cf58b71e3,superpixels are perceptually meaningful atomic...,superpixels perceptually meaningful atomic reg...,superpixels perceptually meaningful atomic reg...
3103,3103,1,This paper focuses on the problem of adaptive ...,"[Zhaoxu Yu, Yan Dong, Shugang Li, Fangfei Li]",0,"[05f79af3-cc32-4214-8d0f-2f7bccf9cae3, 2fb4e72...",Adaptive tracking control for switched strict-...,Neurocomputing,2017,416692e0-e58d-45ac-b392-37942c8b26c9,this paper focuses on the problem of adaptive ...,paper focuses problem adaptive tracking contro...,paper focus problem adaptive tracking control ...
3950,3950,5,Adaptive user-interfaces (AUIs) can enhance th...,"[Harold Soh, Scott Sanner, Madeleine White, Gr...",0,"[0c3ac82d-a897-49a5-bb5d-39bf156dd323, 10efbb3...",Deep Sequential Recommendation for Personalize...,intelligent user interfaces,2017,2ac99316-8801-48f2-9d88-8a51adc0658f,adaptive user-interfaces (auis) can enhance th...,adaptive user-interfaces (auis) enhance usabil...,adaptive user-interfaces (auis) enhance usabil...
3971,3971,1,ABSTRACTThis paper propo

In [32]:
#sub_df.to_csv("Content Filtered Data.csv", index = False)